# Test모드 로그 처리 (추출, 정리, 저장)

In [4]:
import import_ipynb
from entire_logs import *

In [9]:
# test 데이터만 추출
def extract_test_df(df):
    return df[df["where"] == "Test"]

# 선생님별로 df 분할
def divide_per_teacher(df):
    df_list = []
    for teacher in list(set(df["teacher"])):
        sub_df = df[df["teacher"] == teacher]
        df_list.append(sub_df)
    return df_list

# df_list의 각 df에 대해 학생별로 분할
def divide_per_student(df_list):
    df_list2 = []
    for df in df_list:
        for student in list(set(df["student"])):
            sub_df = df[df["student"] == student]
            df_list2.append(sub_df)
    return df_list2

# 단일 df에 대해 try_num column추가
def marking_try_num(df):
    df = df.sort_values(by=["time"]) 
    exit_times = df[df["action"] == "ExitScene"]["time"]
    df_list = []
    for i, exit_time in enumerate(exit_times):
        sub_df = df[df["time"] <= exit_time]
        df = df[exit_time < df["time"]]
        sub_df["try"] = i+1
        
        
        if 1 <= len(sub_df[sub_df["action"] == "EnterScene"]):
            start_time = sub_df[sub_df["action"] == "EnterScene"].iloc[-1].time
#             print(start_time)
            sub_df1 = sub_df[sub_df["time"] < start_time].copy() # 불완전
            sub_df2 = sub_df[start_time <= sub_df["time"]] # 완전
            df_list.append(sub_df2)

            if 0 < len(sub_df1):
                sub_df1["try"] = -(i+1)
                df_list.append(sub_df1)
        else:
            df_list.append(sub_df)
        
    df = pd.concat(df_list)
    return df

# df list의 각 df에 대해 try_num column 추가
def mark_try(df_list):
    df_list2 = []
    for df in df_list:
        try:
            df = marking_try_num(df)
            df_list2.append(df)
        except:
            pass
    return df_list2

# df list의 각 df에 대해 try별로 분할
def divide_per_try(df_list):
    df_list2 = []
    for df in df_list:
        try_list = list(set(df["try"]))
        for try_num in try_list:
                sub_df = df[df["try"] == try_num]
                df_list2.append(sub_df)
    return df_list2

# 
def marking_note_num(df):
    df = df.sort_values(by=["time"])
    split_times = df[df["action"] == "ClearNote"]["time"]
    df_list = []
    for i, split_time in enumerate(split_times):
        sub_df = df[df["time"] <= split_time]
        df = df[split_time < df["time"]]
        sub_df["note_idx"] = i+1
        df_list.append(sub_df)
    
    df = pd.concat(df_list)
    return df

def mark_note(df_list):
    df_list2 = []
    for df in df_list:
        try:
            df = marking_note_num(df)
            df_list2.append(df)
        except:
            pass
    return df_list2

def extract_and_process_test_df(df):
    df = origin_df
    df = extract_test_df(df)
    df_list = divide_per_teacher(df)
    df_list = divide_per_student(df_list)
    df_list = mark_try(df_list)
    df_list = divide_per_try(df_list)
    df_list = mark_note(df_list)
    return pd.concat(df_list)

# 원본 df로 부터 test 로그만 추출, 정리, 저장 후 반환
# cache기능 제공
def get_test_df():
    cache_file_path = "data/processed/processed_test_df.csv"
    if os.path.isfile(cache_file_path):
        return pd.read_csv(cache_file_path, encoding="cp949", index_col = 0)
    
    df = get_entire_df()
    df = extract_and_process_test_df(df)
    df.to_csv(cache_file_path, encoding="cp949")
    return df

In [10]:
# test_df = get_test_df()
# test_df

,time,where,action,option,teacher,student,try,note_idx
30382,2022-11-29-20-08-34-223,Test,EnterScene,"{musicName:여행을 떠나요, pitchAdjust:0, AnswerRange...",최민웅_Log,Test,1,1
30383,2022-11-29-20-08-37-260,Test,StartNote,"{pitch:REST, length:Note8}",최민웅_Log,Test,1,1
30384,2022-11-29-20-08-37-261,Test,Microphone,"{pitch:REST, volume:55.54912}",최민웅_Log,Test,1,1
30385,2022-11-29-20-08-37-264,Test,Microphone,"{pitch:REST, volume:55.54912}",최민웅_Log,Test,1,1
30386,2022-11-29-20-08-37-266,Test,Microphone,"{pitch:REST, volume:55.54912}",최민웅_Log,Test,1,1
...,...,...,...,...,...,...,...,...
159511,2022-12-13-13-14-38-456,Test,Microphone,"{pitch:F3s, volume:26.10585}",선우-이승희,신하은,-5,32
159512,2022-12-13-13-14-38-474,Test,Microphone,"{pitch:F3s, volume:26.10585}",선우-이승희,신하은,-5,32
159513,2022-12-13-13-14-38-490,Test,Microphone,"{pitch:F3s, volume:26.10585}",선우-이승희,신하은,-5,32
159514,2022-12-13-13-14-38-522,Test,ClearNote,{},선우-이승희,신하은,-5,32


In [12]:
# teacher = "성심-김원석(로그)"
# student = "윤예준"
# try_num = 1
# note_idx = 8
# sub_df = test_df[(test_df["teacher"]== teacher) & 
#         (test_df["student"]== student) & 
#         (test_df["try"]== try_num) & 
#         (test_df["note_idx"]== note_idx)]

# sub_df

,time,where,action,option,teacher,student,try,note_idx
68941,2022-12-15-14-09-41-471,Test,StartNote,"{pitch:C4, length:Note8}",성심-김원석(로그),윤예준,1,8
68942,2022-12-15-14-09-41-472,Test,Microphone,"{pitch:G3s, volume:76.12896}",성심-김원석(로그),윤예준,1,8
68943,2022-12-15-14-09-41-480,Test,Microphone,"{pitch:G3s, volume:76.12896}",성심-김원석(로그),윤예준,1,8
68944,2022-12-15-14-09-41-493,Test,Microphone,"{pitch:G3s, volume:76.12896}",성심-김원석(로그),윤예준,1,8
68945,2022-12-15-14-09-41-510,Test,Microphone,"{pitch:G3s, volume:76.12896}",성심-김원석(로그),윤예준,1,8
...,...,...,...,...,...,...,...,...
68998,2022-12-15-14-09-42-394,Test,Microphone,"{pitch:D2s, volume:100.6985}",성심-김원석(로그),윤예준,1,8
68999,2022-12-15-14-09-42-410,Test,Microphone,"{pitch:D2s, volume:100.6985}",성심-김원석(로그),윤예준,1,8
69000,2022-12-15-14-09-42-427,Test,Microphone,"{pitch:D2s, volume:100.6985}",성심-김원석(로그),윤예준,1,8
69001,2022-12-15-14-09-42-445,Test,Microphone,"{pitch:D2s, volume:100.6985}",성심-김원석(로그),윤예준,1,8
